In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(2, 10)
        self.fc2 = nn.Linear(10, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = softmax(x, dim=1, estable=False)
        return x

    
class BatchingDataset():
    def __init__(self, datos, clases, C):
        self.C = C
        self.datos = datos
        self.N = len(self.datos)
        self.clases = torch.FloatTensor(np.eye(self.N, self.C)[clases])
        if torch.cuda.is_available():
            self.datos = self.datos.type(torch.cuda.FloatTensor)
            self.clases = self.clases.type(torch.cuda.FloatTensor)
        return

    def __len__(self):
        return self.N

    def __getitem__(self, i):
        #print(self.Td[0])
        return self.datos[i]

    def paquetes(self, B):
        Tdb = []
        Tcb = []
        d = self.N%B
        if d==0:
            for i in range(0, self.N - 1, B):
                Tdb.append(self.datos[i:i + B])
                Tcb.append(self.clases[i:i + B])
        else:
            for i in range(0, self.N - d, B):
                Tdb.append(self.datos[i:i + B])
                Tcb.append(self.clases[i:i + B])
            Tdb.append(self.datos[i + B:])
            Tcb.append(self.clases[i + B:])
        return zip(Tdb, Tcb)
    
    
def train(net, dataset, optimizer, epochs, minibatches):
    criterion = nn.MSELoss()
    for epoch in range(epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        i = 0
        for x, y in dataset.paquetes(minibatches):
            # get the inputs
            inputs = x
            labels = y
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward + backward + optimize
            outputs = net(inputs)
            loss = cross_ent_loss(outputs, y, epsilon=1e-10) # criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # print statistics
            running_loss += loss.item()
            i += 1
        print('Loss', running_loss/i)       
    print('Finished Training')
    return


def softmax(T, dim, estable=True):
    if not estable:
        m = torch.max(T, dim=dim, keepdim=True)[0].expand_as(T)
        T_m = T - m
        T_out = softmax(T_m, dim=dim, estable=True)
        return T_out
    elif estable:
        T_exp = torch.exp(T)   
        T_sum = torch.sum(T_exp, dim=dim, keepdim=True).expand_as(T)
        T_out = T_exp*torch.reciprocal(T_sum)
        T_out[T_out<1e-5] = 1e-5
        T_out[T_out>1 - 1e-5] = 1 - 1e-5
        if torch.cuda.is_available():
            T_out = T_out.type(torch.cuda.FloatTensor)
        return T_out
    

def cross_ent_loss(Q, P, epsilon=1e-10):
    N = Q.shape[0]
    Q[Q<epsilon] = epsilon
    Q[Q>1 - epsilon] = 1 - epsilon
    return - torch.sum(P.mul(torch.log(Q)))/N

In [3]:
X = torch.FloatTensor(np.array([[0, 0], [0, 1], [1, 0], [1, 1]]))
Y = torch.LongTensor(np.array([0, 1, 1, 0]))

C = 2
dataset = BatchingDataset(X, Y, C=C)

print('X', dataset.datos)
print('clases', dataset.clases)

X tensor([[ 0.,  0.],
        [ 0.,  1.],
        [ 1.,  0.],
        [ 1.,  1.]])
clases tensor([[ 1.,  0.],
        [ 0.,  1.],
        [ 0.,  1.],
        [ 1.,  0.]])


In [4]:
net = Net()
net.zero_grad()
print(net)

# create your optimizer
optimizer = optim.Adam(net.parameters(), lr=1e-2)

Net(
  (fc1): Linear(in_features=2, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=2, bias=True)
)


In [5]:
train(net, dataset, optimizer, epochs=1000, minibatches=2, show_each=2)

Loss 0.6979373395442963
Loss 0.6873706877231598
Loss 0.6809012591838837
Loss 0.6764704287052155
Loss 0.6734790802001953
Loss 0.6706130504608154
Loss 0.6676131784915924
Loss 0.6636938750743866
Loss 0.6598821878433228
Loss 0.6558426320552826
Loss 0.6516440808773041
Loss 0.6474736928939819
Loss 0.643253743648529
Loss 0.6386961936950684
Loss 0.6340575516223907
Loss 0.6290125250816345
Loss 0.6238309144973755
Loss 0.6186474561691284
Loss 0.6133686304092407
Loss 0.6080304682254791
Loss 0.6030366122722626
Loss 0.5974973142147064
Loss 0.5917837917804718
Loss 0.5861423909664154
Loss 0.5805787444114685
Loss 0.5746118128299713
Loss 0.5682806074619293
Loss 0.5631172060966492
Loss 0.55638587474823
Loss 0.5501693338155746
Loss 0.544768750667572
Loss 0.5381458401679993
Loss 0.5316604822874069
Loss 0.5250028818845749
Loss 0.5179801285266876
Loss 0.511701375246048
Loss 0.5051049590110779
Loss 0.4980514943599701
Loss 0.491300106048584
Loss 0.4846945106983185
Loss 0.4778027832508087
Loss 0.470821887254714

Loss 0.008878047578036785
Loss 0.008821832248941064
Loss 0.008765466744080186
Loss 0.008718457771465182
Loss 0.008677069563418627
Loss 0.008629438001662493
Loss 0.008573864120990038
Loss 0.008524381089955568
Loss 0.008482993580400944
Loss 0.008434317074716091
Loss 0.00838547432795167
Loss 0.008338244166225195
Loss 0.008287932723760605
Loss 0.008240605238825083
Loss 0.008196973241865635
Loss 0.008160875644534826
Loss 0.008118528872728348
Loss 0.00807542772963643
Loss 0.00803133542649448
Loss 0.007984193041920662
Loss 0.0079439093824476
Loss 0.007898212410509586
Loss 0.007854873314499855
Loss 0.007823324762284756
Loss 0.007772351615130901
Loss 0.007732800906524062
Loss 0.007696221582591534
Loss 0.007656612433493137
Loss 0.007616066839545965
Loss 0.007572404574602842
Loss 0.007532512303441763
Loss 0.007489615119993687
Loss 0.0074523568619042635
Loss 0.007411235012114048
Loss 0.0073725475231185555
Loss 0.007338583003729582
Loss 0.007302322890609503
Loss 0.007273518247529864
Loss 0.00722616

Loss 0.0023607038310728967
Loss 0.0023537189699709415
Loss 0.0023478425573557615
Loss 0.002341966493986547
Loss 0.0023347852984443307
Loss 0.002326973539311439
Loss 0.0023203172022476792
Loss 0.0023135271621868014
Loss 0.002306020585820079
Loss 0.0022982906666584313
Loss 0.0022936304449103773
Loss 0.0022854272392578423
Loss 0.002279428648762405
Loss 0.002273642341606319
Loss 0.0022665212745778263
Loss 0.002261258428916335
Loss 0.0022533158771693707
Loss 0.002246632124297321
Loss 0.002240230212919414
Loss 0.002233471372164786
Loss 0.0022266380256041884
Loss 0.0022211676696315408
Loss 0.0022147621493786573
Loss 0.0022091526770964265
Loss 0.002202871721237898
Loss 0.0021965187625028193
Loss 0.002190206665545702
Loss 0.0021835825173184276
Loss 0.002177649294026196
Loss 0.0021718157222494483
Loss 0.0021651590359397233
Loss 0.0021586104994639754
Loss 0.0021532493410632014
Loss 0.0021473689121194184
Loss 0.0021409502369351685
Loss 0.0021352226031012833
Loss 0.0021294215694069862
Loss 0.002123

In [18]:
x = torch.ones(2, 2, requires_grad=True)
y = torch.randn(2, 2, requires_grad=True)
z = torch.sum(2*x + y)
z.backward()uo
print(x.grad, y.grad)

tensor([[ 2.,  2.],
        [ 2.,  2.]]) tensor([[ 1.,  1.],
        [ 1.,  1.]])
